이번 과제는 Bert Model을 사용하여 BBC 뉴스 기사의 category를 분류해보는 과제입니다. clone coding을 하시되, 코드 주석을 line by line으로 꼼꼼하게 달아보시며 공부해보세요!

예습과제1: BERT BBC 기사 분류

BERT 모델을 통해 BBC 기사의 카테고리를 분류해보는 과제입니다.
빈칸은 따로 없고, clone coding을 해보시되, 주석을 꼼꼼하게 작성하며 공부해보시길 바랍니다. 아래 파일을 드라이브에 저장해서 코드를 돌려주시기 바랍니다!
- bbc-text.csv

## 데이터 로드 및 탐색

In [1]:
%%capture
!pip install transformers

In [2]:
import pandas as pd
import torch
import numpy as np
from transformers import BertTokenizer, BertModel
from torch import nn
from torch.optim import Adam
from tqdm import tqdm

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
df = pd.read_csv('/content/drive/MyDrive/24-winter KUBIG NLP/WEEK 4/bbc-text.csv')

In [5]:
df.head()
# 인덱스 - 기술 스포츠 등의 카테고리 - 카테고리에 따른 기사 내용

,category,text
0,tech,tv future in the hands of viewers with home th...
1,business,worldcom boss left books alone former worldc...
2,sport,tigers wary of farrell gamble leicester say ...
3,sport,yeading face newcastle in fa cup premiership s...
4,entertainment,ocean s twelve raids box office ocean s twelve...


In [6]:
print(len(df))

2225


In [7]:
df.groupby('category').count()

,text
category,
business,510
entertainment,386
politics,417
sport,511
tech,401


## BertTokenizer

토크나이저로 pretrain된 BERT의 BertTokenizer를 갖고 옵니다. 여러 종류를 시도해보세요.

*파라미터 수, 대소문자 변경 여부 등이 차이남*

- bert-base-uncased : 108MB param, all lowercase
- bert-large-cased : 340MB param, both upper and lower
- bert-base-cased : 108MB param, multi language, both upper and lower


In [8]:
tokenizer = BertTokenizer.from_pretrained('bert-base-cased')
labels = {'business':0,
          'entertainment':1,
          'sport':2,
          'tech':3,
          'politics':4
          }

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/436k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

## Dataset

In [9]:
class Dataset(torch.utils.data.Dataset):

    def __init__(self, df):
        self.labels = [labels[label] for label in df['category']] # 레이블 로드
        self.texts = [tokenizer(text,   # 텍스트 토큰화
                               padding='max_length', max_length = 512, truncation=True,
                                return_tensors="pt") for text in df['text']]

    def classes(self):
        return self.labels

    def __len__(self): # 샘플개수 반환
        return len(self.labels)

    def get_batch_labels(self, idx): # 인덱스 레이블 반환
        return np.array(self.labels[idx])

    def get_batch_texts(self, idx): # 인덱스 텍스트 반환
        return self.texts[idx]

    def __getitem__(self, idx): # 인덱스 샘플 반환

        batch_texts = self.get_batch_texts(idx)
        batch_y = self.get_batch_labels(idx)

        return batch_texts, batch_y

## Train & Evaluate BertClassifier

pretrain된 BertModel을 불러옵니다. 다른 간단한 층들도 같이 쌓아줍니다.

- bert-base-cased: 12-layer, 768-hidden, 12-self attention heads, 110M parameters. Trained on cased English text.


다른 종류들의 pretrianed model은 아래 링크에서 확인할 수 있습니다.

https://huggingface.co/transformers/v2.9.1/pretrained_models.html

In [10]:
class BertClassifier(nn.Module):

    def __init__(self, dropout=0.5):

        super(BertClassifier, self).__init__()

        # 사전 훈련된 모델 로드
        self.bert = BertModel.from_pretrained('bert-base-cased')
        # 오버피팅 방지 위해 드랍아웃 (생성자에서 비율 결정)
        self.dropout = nn.Dropout(dropout)
        # 선형변환
        self.linear = nn.Linear(768, 5)
        # 활성화 함수
        self.relu = nn.ReLU()

    def forward(self, input_id, mask): # input_id 는 텍스트의 인덱스, mask 는 어텐션 적용할 위치

        # 버트 통과 (토큰화 및 인덱스화된 텍스트, 마스크 등을 적용)
        _, pooled_output = self.bert(input_ids= input_id, attention_mask=mask,return_dict=False)
        # 과적합 방지
        dropout_output = self.dropout(pooled_output)
        # 선형변환
        linear_output = self.linear(dropout_output)
        # 활성화 함수
        final_layer = self.relu(linear_output)

        return final_layer

In [11]:
def train(model, train_data, val_data, learning_rate, epochs):

    # 훈련함수 지정

    train, val = Dataset(train_data), Dataset(val_data)

    train_dataloader = torch.utils.data.DataLoader(train, batch_size=2, shuffle=True)
    val_dataloader = torch.utils.data.DataLoader(val, batch_size=2)

    # gpu 확인
    use_cuda = torch.cuda.is_available()
    device = torch.device("cuda" if use_cuda else "cpu")

    # 매개변수 지정
    criterion = nn.CrossEntropyLoss()
    optimizer = Adam(model.parameters(), lr= learning_rate)

    if use_cuda:

            model = model.cuda()
            criterion = criterion.cuda()




    for epoch_num in range(epochs):

            # 훈련 데이터 손실 및 정확도 초기화
            total_acc_train = 0
            total_loss_train = 0

            # 훈련 시작
            for train_input, train_label in tqdm(train_dataloader):

                train_label = train_label.to(device)
                # 어텐션 마스크도 gpu로 보내 bert 모델이 데이터/패딩을 구분 하게 함
                mask = train_input['attention_mask'].to(device)
                input_id = train_input['input_ids'].squeeze(1).to(device)
                # 예측 수행
                output = model(input_id, mask)

                # 손실 계산
                batch_loss = criterion(output, train_label.long())
                total_loss_train += batch_loss.item()

                # 정확도 계산
                acc = (output.argmax(dim=1) == train_label).sum().item()
                total_acc_train += acc

                # 파라미터 업데이트
                model.zero_grad()
                batch_loss.backward()
                optimizer.step()


            # 검증 데이터 손실 및 정확도 초기화
            total_acc_val = 0
            total_loss_val = 0

            # 경사 업데이트 없이, 성능평가
            with torch.no_grad():

                for val_input, val_label in val_dataloader:

                    val_label = val_label.to(device)
                    mask = val_input['attention_mask'].to(device)
                    input_id = val_input['input_ids'].squeeze(1).to(device)

                    output = model(input_id, mask)

                    # 비슷한 과정을 거치나, 그레디언트 업데이트 안함

                    batch_loss = criterion(output, val_label.long())
                    total_loss_val += batch_loss.item()

                    acc = (output.argmax(dim=1) == val_label).sum().item()
                    total_acc_val += acc

            print(
                f'Epochs: {epoch_num + 1} | Train Loss: {total_loss_train / len(train_data): .3f} | Train Accuracy: {total_acc_train / len(train_data): .3f} | Val Loss: {total_loss_val / len(val_data): .3f} | Val Accuracy: {total_acc_val / len(val_data): .3f}')


In [12]:
def evaluate(model, test_data):

    # 평가
    test = Dataset(test_data)

    test_dataloader = torch.utils.data.DataLoader(test, batch_size=2)

    use_cuda = torch.cuda.is_available()
    device = torch.device("cuda" if use_cuda else "cpu")

    if use_cuda:

        model = model.cuda()

    total_acc_test = 0
    with torch.no_grad():

        for test_input, test_label in test_dataloader:

              test_label = test_label.to(device)
              mask = test_input['attention_mask'].to(device)
              input_id = test_input['input_ids'].squeeze(1).to(device)

                # 마찬가지로 경사 업데이트 X
              output = model(input_id, mask)

              acc = (output.argmax(dim=1) == test_label).sum().item()
              total_acc_test += acc

    print(f'Test Accuracy: {total_acc_test / len(test_data): .3f}')

In [13]:
np.random.seed(112) # 난수 생성 시드 설정

# 훈련, 검증, 테스트 세트로 데이터를 무작위로 섞어 샘플링
# random_state 는 재현성 위해 사용
# 80% 를 train / 10 % val / 10% test

df_train, df_val, df_test = np.split(df.sample(frac=1, random_state=42),
                                     [int(.8*len(df)), int(.9*len(df))])

print(len(df_train),len(df_val), len(df_test))

1780 222 223


In [14]:
EPOCHS = 2 #EPOCH 수 늘려보기!
model = BertClassifier() # bert-base cased 사용
LR = 1e-6

train(model, df_train, df_val, LR, EPOCHS)

model.safetensors:   0%|          | 0.00/436M [00:00<?, ?B/s]

100%|██████████| 890/890 [03:05<00:00,  4.79it/s]


Epochs: 1 | Train Loss:  0.768 | Train Accuracy:  0.341 | Val Loss:  0.588 | Val Accuracy:  0.667


100%|██████████| 890/890 [03:21<00:00,  4.42it/s]


Epochs: 2 | Train Loss:  0.355 | Train Accuracy:  0.872 | Val Loss:  0.167 | Val Accuracy:  0.986


In [15]:
evaluate(model, df_test)

Test Accuracy:  0.982


## 에폭 늘리기

In [16]:
EPOCHS = 5 #EPOCH 수 늘려보기!
model = BertClassifier() # bert-base cased 사용
LR = 1e-6

train(model, df_train, df_val, LR, EPOCHS)

100%|██████████| 890/890 [03:21<00:00,  4.43it/s]


Epochs: 1 | Train Loss:  0.733 | Train Accuracy:  0.408 | Val Loss:  0.572 | Val Accuracy:  0.698


100%|██████████| 890/890 [03:22<00:00,  4.39it/s]


Epochs: 2 | Train Loss:  0.370 | Train Accuracy:  0.874 | Val Loss:  0.210 | Val Accuracy:  0.973


100%|██████████| 890/890 [03:23<00:00,  4.38it/s]


Epochs: 3 | Train Loss:  0.162 | Train Accuracy:  0.975 | Val Loss:  0.097 | Val Accuracy:  0.995


100%|██████████| 890/890 [03:22<00:00,  4.39it/s]


Epochs: 4 | Train Loss:  0.080 | Train Accuracy:  0.987 | Val Loss:  0.063 | Val Accuracy:  0.991


100%|██████████| 890/890 [03:22<00:00,  4.39it/s]


Epochs: 5 | Train Loss:  0.046 | Train Accuracy:  0.994 | Val Loss:  0.040 | Val Accuracy:  0.995


In [17]:
evaluate(model, df_test)

Test Accuracy:  0.987


## 다른 토크나이저 사용

In [18]:
tokenizer_un = BertTokenizer.from_pretrained('bert-base-uncased')

tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [19]:
class Dataset(torch.utils.data.Dataset):

    def __init__(self, df):
        self.labels = [labels[label] for label in df['category']] # 레이블 로드
        self.texts = [tokenizer_un(text,   # 텍스트 토큰화
                               padding='max_length', max_length = 512, truncation=True,
                                return_tensors="pt") for text in df['text']]

    def classes(self):
        return self.labels

    def __len__(self): # 샘플개수 반환
        return len(self.labels)

    def get_batch_labels(self, idx): # 인덱스 레이블 반환
        return np.array(self.labels[idx])

    def get_batch_texts(self, idx): # 인덱스 텍스트 반환
        return self.texts[idx]

    def __getitem__(self, idx): # 인덱스 샘플 반환

        batch_texts = self.get_batch_texts(idx)
        batch_y = self.get_batch_labels(idx)

        return batch_texts, batch_y

In [20]:
class BertClassifier(nn.Module):

    def __init__(self, dropout=0.5):

        super(BertClassifier, self).__init__()

        # 사전 훈련된 모델 로드
        self.bert = BertModel.from_pretrained('bert-base-uncased')
        # 오버피팅 방지 위해 드랍아웃 (생성자에서 비율 결정)
        self.dropout = nn.Dropout(dropout)
        # 선형변환
        self.linear = nn.Linear(768, 5)
        # 활성화 함수
        self.relu = nn.ReLU()

    def forward(self, input_id, mask): # input_id 는 텍스트의 인덱스, mask 는 어텐션 적용할 위치

        # 버트 통과 (토큰화 및 인덱스화된 텍스트, 마스크 등을 적용)
        _, pooled_output = self.bert(input_ids= input_id, attention_mask=mask,return_dict=False)
        # 과적합 방지
        dropout_output = self.dropout(pooled_output)
        # 선형변환
        linear_output = self.linear(dropout_output)
        # 활성화 함수
        final_layer = self.relu(linear_output)

        return final_layer

In [21]:
def train(model, train_data, val_data, learning_rate, epochs):

    # 훈련함수 지정

    train, val = Dataset(train_data), Dataset(val_data)

    train_dataloader = torch.utils.data.DataLoader(train, batch_size=2, shuffle=True)
    val_dataloader = torch.utils.data.DataLoader(val, batch_size=2)

    # gpu 확인
    use_cuda = torch.cuda.is_available()
    device = torch.device("cuda" if use_cuda else "cpu")

    # 매개변수 지정
    criterion = nn.CrossEntropyLoss()
    optimizer = Adam(model.parameters(), lr= learning_rate)

    if use_cuda:

            model = model.cuda()
            criterion = criterion.cuda()



    for epoch_num in range(epochs):

            # 훈련 데이터 손실 및 정확도 초기화
            total_acc_train = 0
            total_loss_train = 0

            # 훈련 시작
            for train_input, train_label in tqdm(train_dataloader):

                train_label = train_label.to(device)
                # 어텐션 마스크도 gpu로 보내 bert 모델이 데이터/패딩을 구분 하게 함
                mask = train_input['attention_mask'].to(device)
                input_id = train_input['input_ids'].squeeze(1).to(device)
                # 예측 수행
                output = model(input_id, mask)

                # 손실 계산
                batch_loss = criterion(output, train_label.long())
                total_loss_train += batch_loss.item()

                # 정확도 계산
                acc = (output.argmax(dim=1) == train_label).sum().item()
                total_acc_train += acc

                # 파라미터 업데이트
                model.zero_grad()
                batch_loss.backward()
                optimizer.step()


            # 검증 데이터 손실 및 정확도 초기화
            total_acc_val = 0
            total_loss_val = 0

            # 경사 업데이트 없이, 성능평가
            with torch.no_grad():

                for val_input, val_label in val_dataloader:

                    val_label = val_label.to(device)
                    mask = val_input['attention_mask'].to(device)
                    input_id = val_input['input_ids'].squeeze(1).to(device)

                    output = model(input_id, mask)

                    # 비슷한 과정을 거치나, 그레디언트 업데이트 안함

                    batch_loss = criterion(output, val_label.long())
                    total_loss_val += batch_loss.item()

                    acc = (output.argmax(dim=1) == val_label).sum().item()
                    total_acc_val += acc

            print(
                f'Epochs: {epoch_num + 1} | Train Loss: {total_loss_train / len(train_data): .3f} | Train Accuracy: {total_acc_train / len(train_data): .3f} | Val Loss: {total_loss_val / len(val_data): .3f} | Val Accuracy: {total_acc_val / len(val_data): .3f}')


In [22]:
def evaluate(model, test_data):

    # 평가
    test = Dataset(test_data)

    test_dataloader = torch.utils.data.DataLoader(test, batch_size=2)

    use_cuda = torch.cuda.is_available()
    device = torch.device("cuda" if use_cuda else "cpu")

    if use_cuda:

        model = model.cuda()

    total_acc_test = 0
    with torch.no_grad():

        for test_input, test_label in test_dataloader:

              test_label = test_label.to(device)
              mask = test_input['attention_mask'].to(device)
              input_id = test_input['input_ids'].squeeze(1).to(device)

                # 마찬가지로 경사 업데이트 X
              output = model(input_id, mask)

              acc = (output.argmax(dim=1) == test_label).sum().item()
              total_acc_test += acc

    print(f'Test Accuracy: {total_acc_test / len(test_data): .3f}')

In [23]:
EPOCHS = 2
model = BertClassifier() # bert-base uncased 사용
LR = 1e-6

train(model, df_train, df_val, LR, EPOCHS)

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

100%|██████████| 890/890 [03:19<00:00,  4.45it/s]


Epochs: 1 | Train Loss:  0.770 | Train Accuracy:  0.352 | Val Loss:  0.680 | Val Accuracy:  0.550


100%|██████████| 890/890 [03:22<00:00,  4.40it/s]


Epochs: 2 | Train Loss:  0.538 | Train Accuracy:  0.771 | Val Loss:  0.399 | Val Accuracy:  0.928


In [24]:
evaluate(model, df_test)

Test Accuracy:  0.937
